In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import requests

In [ ]:
# This is the website where we pull the information for 2021 new releases
url = 'https://en.wikipedia.org/wiki/List_of_American_films_of_2021'

# These headers will allow us to avoid a 403 error by mimicing a web browser
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [ ]:
# format the request to mimic a web browser
r = requests.get(url, headers=header)

# import tables using pandas
tables = pd.read_html(r.text)

In [ ]:
# read each of the quarterly tables into dataframes

q1 = tables[1]
q2 = tables[2]
q3 = tables[3]
q4 = tables[4]

In [ ]:
# concatenate each of the quarterly movie tables into one dataframe
all_quarters = [q1, q2, q3, q4]

future_movies = pd.concat(all_quarters)

In [ ]:
future_movies.shape
# future_movies.head()

In [ ]:
# This is the website where we pull the information for 2021 new releases
new_url = 'https://www.imdb.com/list/ls093883812/?sort=list_order,asc&st_dt=&mode=detail&page=1&ref_=ttls_vm_dtl'

# These headers will allow us to avoid a 403 error by mimicing a web browser
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [ ]:
# format the request to mimic a web browser
r2 = requests.get(new_url, headers=header)

# import tables using pandas
newtables = pd.read_html(r2.text)

In [ ]:
newtables[1]